In [16]:
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import LabelEncoder

In [ ]:
# NOTE: Custome Module !!
from module.dataframe import df_query_by_type, series_string_filter
from module.preprocess import do_preprocessing

---

In [19]:
train = pd.read_parquet("./database/train.parquet")
test = pd.read_parquet("./database/test.parquet")

In [20]:
train, test = do_preprocessing(train, test)

In [21]:
train_x, train_y = train.drop(columns="target"), train["target"]
test_x = test

---

In [22]:
# MAX FEATURE!!!
CAT = ["day_of_week",
"weight_restricted",
"road_type",
"start_turn_restricted",
"end_turn_restricted",
"is_holiday",]
train_x = pd.get_dummies(train_x,columns=CAT)
test_x = pd.get_dummies(test_x,columns=CAT)

---

In [23]:
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from lightgbm import LGBMRegressor

In [24]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
eval_set = [(x_valid, y_valid)]

model_params = {
    "objective": "regression",
    "boosting_type": "gbdt",
    "metric": "rmse",
    "n_estimators": 8000,
    "learning_rate": 0.003,
    "feature_fraction": 0.8,
    "reg_alpha": 0.3,
    "reg_lambda": 0.1,
    "subsample": 0.6,
    "colsample_bytree": 0.8,
    "verbose": 1,
    "early_stopping_rounds": 200,
}
regress_model = LGBMRegressor(
    **model_params,
)
regress_model.fit(
    x_train,
    y_train,
    eval_metric="rmse",
    eval_set=eval_set,
)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.096397 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 3760973, number of used features: 28
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200


LGBMRegressor(colsample_bytree=0.8, early_stopping_rounds=200,
              feature_fraction=0.8, learning_rate=0.003, metric='rmse',
              n_estimators=8000, objective='regression', reg_alpha=0.3,
              reg_lambda=0.1, subsample=0.6, verbose=1)

In [25]:
# Eval
from sklearn.metrics import r2_score as R2
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import median_absolute_error as MAE

MAE(regress_model.predict(x_valid), y_valid), R2(regress_model.predict(x_valid), y_valid),RMSE(regress_model.predict(x_valid), y_valid)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8


(5.9163635572803415, 0.3968929932639398, 9.69793504390368)

## Submission
---

In [26]:
sample_submission = pd.read_csv('./database/sample_submission.csv')

In [27]:
sample_submission["target"] = regress_model.predict(test_x)
sample_submission.to_csv("./submit_lgb.csv", index = False)
sample_submission

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.8


,id,target
0,TEST_000000,24.064434
1,TEST_000001,45.916051
2,TEST_000002,50.593115
3,TEST_000003,37.628754
4,TEST_000004,39.549292
...,...,...
291236,TEST_291236,48.515529
291237,TEST_291237,38.154096
291238,TEST_291238,22.212678
291239,TEST_291239,26.417652
